In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [8]:
c_data=pd.read_csv('country_data',  encoding='latin-1')
#Filter to US only
us_data=c_data[c_data['geoId']=='US']
sort_us=us_data.sort_values(by=['year','month', 'day'])
sort_us= sort_us[['day', 'month', 'year', 'cases', 'deaths', 'geoId', 'popData2018']]
sort_us

,day,month,year,cases,deaths,geoId,popData2018
6982,31,12,2019,0,0,US,327167434.0
6981,1,1,2020,0,0,US,327167434.0
6980,2,1,2020,0,0,US,327167434.0
6979,3,1,2020,0,0,US,327167434.0
6978,4,1,2020,0,0,US,327167434.0
...,...,...,...,...,...,...,...
6899,23,3,2020,8459,131,US,327167434.0
6898,24,3,2020,11236,119,US,327167434.0
6897,25,3,2020,8789,211,US,327167434.0
6896,26,3,2020,13963,249,US,327167434.0


In [16]:
c_data=pd.read_csv('us-counties.csv',  encoding='latin-1')
#Filter to US only
c_data[c_data['county'] == 'Snohomish']

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0
7,2020-01-25,Snohomish,Washington,53061.0,1,0
...,...,...,...,...,...,...
7064,2020-03-21,Snohomish,Washington,53061.0,446,9
8201,2020-03-22,Snohomish,Washington,53061.0,480,10
9427,2020-03-23,Snohomish,Washington,53061.0,518,10
10779,2020-03-24,Snohomish,Washington,53061.0,614,16


In [21]:
c_data=pd.read_csv('us-states.csv',  encoding='latin-1')
#Filter to US only
c_data

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
1276,2020-03-25,Virginia,51,391,9
1277,2020-03-25,Washington,53,2585,130
1278,2020-03-25,West Virginia,54,52,0
1279,2020-03-25,Wisconsin,55,623,7


In [32]:
c_data=pd.read_csv('with_Counties.csv',  encoding='latin-1')
#Filter to US only
c_data

,zip,primary_city,state,latitude,longitude,county,type,world_region,country,decommissioned,estimated_population,notes
0,501,Holtsville,NY,40.81,-73.04,Suffolk,UNIQUE,NaN,US,0,384,NaN
1,544,Holtsville,NY,40.81,-73.04,Suffolk,UNIQUE,NaN,US,0,0,NaN
2,601,Adjuntas,PR,18.16,-66.72,Adjuntas,STANDARD,NaN,US,0,0,NaN
3,602,Aguada,PR,18.38,-67.18,NaN,STANDARD,NaN,US,0,0,NaN
4,603,Aguadilla,PR,18.43,-67.15,Aguadilla,STANDARD,NaN,US,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
42517,99926,Metlakatla,AK,55.14,-131.49,Prince of Wales-Outer Ketchikan Borough,PO BOX,NaN,US,0,1018,NaN
42518,99927,Point Baker,AK,56.30,-133.57,Prince of Wales-Outer Ketchikan Borough,PO BOX,NaN,US,0,0,NaN
42519,99928,Ward Cove,AK,55.45,-131.79,Ketchikan Gateway Borough,PO BOX,NaN,US,0,1281,NaN
42520,99929,Wrangell,AK,55.95,-131.96,Wrangell Borough,PO BOX,NaN,US,0,1956,NaN


In [24]:
!pip install geopy

     |████████████████████████████████| 104 kB 1.9 MB/s eta 0:00:01


In [28]:
import geopy.distance

coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)



279.35290160430094

In [31]:
%%timeit 
geopy.distance.geodesic(coords_1, coords_2).km

528 µs ± 93.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [43]:
c_data[['latitude', 'longitude']]

,latitude,longitude
0,40.81,-73.04
1,40.81,-73.04
2,18.16,-66.72
3,18.38,-67.18
4,18.43,-67.15
...,...,...
42517,55.14,-131.49
42518,56.30,-133.57
42519,55.45,-131.79
42520,55.95,-131.96


In [50]:
d = geopy.distance.geodesic
def f(x):
    return d(x, (52.2296756, 21.0122287))

In [52]:
%time c_data[['latitude', 'longitude']].apply(lambda x: f(x), axis=1)

CPU times: user 20.5 s, sys: 289 ms, total: 20.7 s
Wall time: 21.9 s


0        6808.454879381718 km
1        6808.454879381718 km
2         8280.13246943454 km
3        8292.302685373757 km
4         8286.01646227181 km
                 ...         
42517    7835.609930892542 km
42518    7750.013685353048 km
42519    7808.377544419615 km
42520    7758.406211285016 km
42521    7817.228653850341 km
Length: 42522, dtype: object

In [35]:
geopy.distance.geodesic((52.2296756, 21.0122287), (c_data['latitude'], c_data['longitude'])).km

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().